In [179]:
class vertex():
    def __init__(self,name):
        self.name=name
        self.cost=0
        self.prev=None
        self.adjacent=[]
    def __repr__(self):
        return "vertex({})".format(self.name)
    def __eq__(self,another):
        if isinstance(another,vertex):
            if another.name==self.name:
                return True
        return False
    #here less than is when the cost of the element is the largest
    def __lt__(self,another):
        return self.cost>another.cost
    def __le__(self,another):
        return self.cost>=another.cost
#here this self.start is not the vertex, but rather the name of ther vertex     
class edge():
    def __init__(self,start_name,end_name,flow,capacity):
        self.start=start_name
        self.end=end_name
        self.flow=flow
        self.capacity=capacity
    def __repr__(self):
        return "edge({},{},{},{})".format(str(self.start),str(self.end),self.flow,self.capacity)
    def __eq__(self,another):
        if isinstance(another,edge):
            if another.start==self.start and another.end==self.end:
                return True
        return False

In [329]:
import heapq
class my_graph():
    def __init__(self):
        self.vertices_name=set()
        self.vertices=[]
        self.edges=[]
    def __repr__(self):
        output=""
        for a_vertex in self.vertices:
            output+=str(a_vertex)+"\n"
        for an_edge in self.edges:
            output+=str(an_edge)+"\n"
        return output
    def get_index(self,name_of_a_vertex):
        return self.vertices.index(vertex(name_of_a_vertex))
    def get_vertex(self,name_of_a_vertex):
        return self.vertices[self.get_index(name_of_a_vertex)]
    def add_edge(self,an_edge):
        self.edges.append(an_edge)
        a=an_edge.start
        b=an_edge.end
        if (a not in self.vertices_name):
            self.vertices_name.add(a)
            self.vertices.append(vertex(a))
        if (b not in self.vertices_name):
            self.vertices_name.add(b)
            self.vertices.append(vertex(b))
        self.get_vertex(a).adjacent.append(an_edge)
    def update_edge(self,start,end,amount_to_add):
        e_forward=edge(start,end,0,0)
        e_reverse=edge(end,start,0,0)
        for an_edge in self.edges:
            if an_edge==e_forward:
                an_edge.flow+=amount_to_add
            if an_edge==e_reverse:
                e=e_reverse
                an_edge.flow-=amount_to_add
#         start_vertex=self.get_vertex(start)
#         for an_edge in start_vertex.adjacent:
#             if an_edge==e:
#                 if forward==True:
#                     an_edge.flow+=amount_to_add
#                 else:
#                     an_edge.flow-=amount_to_add
    def maxcost(self,a_vertex_name):
        for vertex in self.vertices:
            vertex.cost=0
            vertex.prev=None
        start=self.get_vertex(a_vertex_name)
        start.cost=100
        visited=set()
        candidates=[]
        candidates.append(start)
        while(len(candidates)>0):
            heapq.heapify(candidates)
            current=heapq.heappop(candidates)
            if current.name not in visited:
                visited.add(current.name)
                for cur_edge in current.adjacent:
                    target=self.get_vertex(cur_edge.end)
                    cur_cost=min(current.cost,cur_edge.flow)
                    if target.cost<cur_cost:
                        target.cost=cur_cost
                        target.prev=current
                        candidates.append(target)
    def find_residue(self):
        num=len(self.vertices)
        matrix=np.zeros([num,num]).astype(int)
        S=self.get_index("source")
        T=self.get_index("sink")
        for e in self.edges:
            start=self.get_index(e.start)
            end=self.get_index(e.end)
            matrix[start][end]=e.capacity-e.flow
            matrix[end][start]=e.flow
        residual_cost,residual_path=find_min_path_matrix(matrix)
        return residual_cost[S][T],residual_path[(S,T)]
    def max_flow(self):
        residue,residual_path=self.find_residue()
        while (residue>0):
            for i in range(len(residual_path)-1):
                cur_vertex=self.vertices[residual_path[i]]
                next_vertex=self.vertices[residual_path[i+1]]
                self.update_edge(cur_vertex.name,next_vertex.name,residue)
            residue,residual_path=self.find_residue()
        final_flow=0
        S=self.get_index("source")
        for edge in self.vertices[S].adjacent:
            final_flow+=edge.flow
        return final_flow
            

In [330]:
def find_min_path_graph(matrix):
    g=my_graph()
    m=len(matrix)
    for i in range(m):
        for j in range(m):
            if i!=j:
                e=edge(i,j,matrix[i][j],0)
                g.add_edge(e)
    for i in range(m):
        g.maxcost(i)
        print("currently on vertex:"+str(i))
        for vertex in g.vertices:
            if vertex.name==i:
                continue
            cost=vertex.cost
            path=[]
            path.append(vertex)
            while vertex.prev!=None and vertex.prev.name!=i:
                vertex=vertex.prev
                path.append(vertex)
            path.append(g.get_vertex(i))
            path.append(cost)
            path.reverse()
            print(path)
#here the input matrix has M[i][j] equals the "cost" of path from vertex i to vertx j
#it outputs i)the max "cost" from i to j and ii)the actual path. We want to maximize the cost because 
#we need this function on the residual network
def find_min_path_matrix(matrix):
    path={}
    m=len(matrix)
    for i in range(m):
        for j in range(m):
            path[(i,j)]=[i]
    for k in range (m):
        for i in range (m):
            for j in range (m):
                if (i==j):
                    continue
                current=min(matrix[i][k],matrix[k][j])
                if matrix[i][j]<current:
                    matrix[i][j]=current
                    path[(i,j)]=[]
                    for q in path[(i,k)]:
                        path[(i,j)].append(q)
                    for q in path[(k,j)]:
                        path[(i,j)].append(q)
    for i in range (m):
        for j in range (m):
            path[(i,j)].append(j)
    return matrix,path


In [331]:
import random
import numpy as np
def generate_random_test(num):
    matrix=[]
    for i in range(num):
        data=[0]*num
        matrix.append(data)
        for j in range(num):
            if i!=j:
                matrix[i][j]=random.randint(0,10)
    print(matrix)
generate_random_test(5)
test=[[0,1,0],[1,0,0],[1,2,0]]
find_min_path_graph(test)
find_min_path_matrix(test)

[[0, 10, 7, 4, 1], [4, 0, 4, 8, 0], [8, 9, 0, 9, 4], [0, 9, 4, 0, 9], [6, 4, 7, 6, 0]]
currently on vertex:0
[1, vertex(0), vertex(1)]
[0, vertex(0), vertex(2)]
currently on vertex:1
[1, vertex(1), vertex(0)]
[0, vertex(1), vertex(2)]
currently on vertex:2
[1, vertex(2), vertex(0)]
[2, vertex(2), vertex(1)]


([[0, 1, 0], [1, 0, 0], [1, 2, 0]],
 {(0, 0): [0, 0],
  (0, 1): [0, 1],
  (0, 2): [0, 2],
  (1, 0): [1, 0],
  (1, 1): [1, 1],
  (1, 2): [1, 2],
  (2, 0): [2, 0],
  (2, 1): [2, 1],
  (2, 2): [2, 2]})

In [338]:
def create_match_set(match_table,cutoff):
    num_students=len(match_table)
    num_mentors=len(match_table[0])
    current_match=np.zeros([num_students,num_mentors]).astype(int)
    for i in range(num_students):
        for j in range(num_mentors):
            if match_table[i][j]>cutoff:
                current_match[i][j]=1
    return current_match

def find_match(match_table,mentor_preference,cutoff):
    num_students=len(match_table)
    num_mentors=len(match_table[0])
    current_set=create_match_set(match_table,cutoff)
    g=my_graph()
    for i in range(num_students):
        for j in range(num_mentors):
            if current_set[i][j]==1:
                g.add_edge(edge("student"+str(i),"mentor"+str(j),0,1))
    for i in range(num_students):
        g.add_edge(edge("source","student"+str(i),0,1))
    for j in range(num_mentors):
        g.add_edge(edge("mentor"+str(j),"sink",0,mentor_preference[j]))
    return g.max_flow(),g

def find_match_all_paired(match_table,mentor_preference,cutoff):
    all_matched=False
    num_students=len(match_table)
    current_flow,current_graph=find_match(match_table,mentor_preference,cutoff)
    while (all_matched==False):
        if current_flow==num_students:
            all_matched=True
        else:
            if(cutoff<0):
                print("No possible match")
                return 
            cutoff-=1
            current_flow,current_graph=find_match(match_table,mentor_preference,cutoff)
    match=[]
    for edge in current_graph.edges:
        if edge.flow!=0 and edge.start!="source" and edge.end!="sink":
            match.append([edge.start,edge.end])
    return match

In [344]:
example_table=[[0,3],[2,1],[2,0],[2,3]]
example_preference=[2,2]
#cutoff score at 2 results in only two mentees being paired
#while changing mentor preference to 1 means not all students are paired
find_match(example_table,example_preference,2)[0]

find_match_all_paired(example_table,example_preference,3)

[['student0', 'mentor1'],
 ['student1', 'mentor0'],
 ['student2', 'mentor0'],
 ['student3', 'mentor1']]

2
